#Standard Fine Tuning

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")
# !pip install datasets


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
raw_datasets


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets["train"]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})

In [ ]:
print(tokenized_datasets["train"][0]["attention_mask"])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(small_train_dataset["input_ids"][0])

[101, 1247, 1110, 1185, 6796, 1120, 1155, 1206, 3144, 2852, 1105, 26890, 1197, 1133, 1103, 1864, 1115, 1241, 1132, 2021, 1326, 1164, 5973, 6969, 119, 26890, 1197, 2736, 19501, 1183, 117, 3144, 2852, 2736, 5263, 119, 26890, 1197, 15836, 1132, 2385, 3014, 119, 3144, 2852, 112, 188, 4928, 1132, 1677, 1167, 8277, 119, 119, 119, 3144, 2852, 2736, 1167, 1176, 3460, 15463, 20629, 117, 1191, 1195, 1138, 1106, 3205, 12672, 119, 119, 119, 1109, 1514, 1959, 1110, 4780, 1105, 6994, 1186, 117, 1133, 1138, 107, 172, 20293, 12716, 3923, 107, 119, 2563, 1176, 1106, 14133, 117, 1106, 3942, 117, 1106, 17459, 119, 1731, 1164, 1198, 8965, 136, 16819, 1645, 1315, 117, 1234, 2269, 3144, 2852, 2736, 1237, 1133, 117, 1113, 1103, 1168, 1289, 117, 8995, 1152, 9353, 1237, 1326, 113, 106, 106, 106, 114, 119, 2389, 1122, 112, 188, 1103, 1846, 117, 1137, 1103, 4840, 117, 1133, 146, 1341, 1142, 1326, 1110, 1167, 1483, 1190, 1237, 119, 1650, 1103, 1236, 117, 1103, 5681, 1132, 1541, 1363, 1105, 6276, 119, 1109, 3176, 

In [ ]:
tf_train_dataset = small_train_dataset.remove_columns(["text"]).with_format("tensorflow")
tf_eval_dataset = small_eval_dataset.remove_columns(["text"]).with_format("tensorflow")

In [ ]:
print(tokenizer.model_input_names)

['input_ids', 'token_type_ids', 'attention_mask']


In [ ]:
tf_train_dataset

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [ ]:
print(type(tf_train_dataset["input_ids"]))

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [ ]:
train_features = {x: tf_train_dataset[x] for x in tokenizer.model_input_names}

In [ ]:
train_features

{'input_ids': <tf.Tensor: shape=(1000, 512), dtype=int64, numpy=
 array([[ 101, 1247, 1110, ...,    0,    0,    0],
        [ 101, 1188, 2523, ...,    0,    0,    0],
        [ 101, 1667,  153, ...,    0,    0,    0],
        ...,
        [ 101, 8762, 1126, ...,    0,    0,    0],
        [ 101, 1109, 1646, ...,    0,    0,    0],
        [ 101,  146, 1138, ...,  117, 1176,  102]])>,
 'token_type_ids': <tf.Tensor: shape=(1000, 512), dtype=int64, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])>,
 'attention_mask': <tf.Tensor: shape=(1000, 512), dtype=int64, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 1, 1, 1]])>}

In [ ]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["label"]))

In [ ]:
for i in train_tf_dataset:
  print(i[1])

tf.Tensor([0 0 1 0 0 0 0 0], shape=(8,), dtype=int64)
tf.Tensor([0 0 1 1 1 0 1 0], shape=(8,), dtype=int64)
tf.Tensor([1 0 0 1 0 1 1 1], shape=(8,), dtype=int64)
tf.Tensor([0 0 1 0 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([1 1 0 1 1 1 0 1], shape=(8,), dtype=int64)
tf.Tensor([0 1 0 0 1 1 0 1], shape=(8,), dtype=int64)
tf.Tensor([1 0 1 1 1 1 1 0], shape=(8,), dtype=int64)
tf.Tensor([1 1 1 0 1 0 0 0], shape=(8,), dtype=int64)
tf.Tensor([1 0 0 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([1 0 0 1 0 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([0 1 1 0 0 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([0 0 0 1 1 0 0 0], shape=(8,), dtype=int64)
tf.Tensor([1 1 1 0 1 1 1 1], shape=(8,), dtype=int64)
tf.Tensor([0 0 0 0 1 1 0 1], shape=(8,), dtype=int64)
tf.Tensor([1 0 1 1 0 1 0 1], shape=(8,), dtype=int64)
tf.Tensor([0 1 0 0 1 1 0 1], shape=(8,), dtype=int64)
tf.Tensor([0 1 0 0 0 0 1 1], shape=(8,), dtype=int64)
tf.Tensor([1 1 0 0 1 0 1 0], shape=(8,), dtype=int64)
tf.Tensor([1 1 0 0 1 0 0 1],

In [ ]:
  train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

In [ ]:
eval_features = {x: tf_eval_dataset[x] for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["label"]))
eval_tf_dataset = eval_tf_dataset.batch(8)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=3)

Epoch 1/3
125/125 [==============================] - 210s 1s/step - loss: 0.6115 - sparse_categorical_accuracy: 0.6530 - val_loss: 0.4066 - val_sparse_categorical_accuracy: 0.8350
Epoch 2/3
125/125 [==============================] - 161s 1s/step - loss: 0.3092 - sparse_categorical_accuracy: 0.8800 - val_loss: 0.3907 - val_sparse_categorical_accuracy: 0.8500
Epoch 3/3
125/125 [==============================] - 202s 2s/step - loss: 0.1981 - sparse_categorical_accuracy: 0.9350 - val_loss: 0.4714 - val_sparse_categorical_accuracy: 0.8210


In [ ]:
from transformers import AutoModelForSequenceClassification

model.save_pretrained("my_imdb_model")
pytorch_model = AutoModelForSequenceClassification.from_pretrained("my_imdb_model", from_tf=True)

All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [ ]:
eval_results = model.evaluate(eval_tf_dataset)
print(f"Evaluation Loss: {eval_results[0]}, Evaluation Accuracy: {eval_results[1]}")

125/125 [==============================] - 41s 329ms/step - loss: 0.4714 - sparse_categorical_accuracy: 0.8210
Evaluation Loss: 0.47142279148101807, Evaluation Accuracy: 0.8209999799728394


Comparison

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Input text
input_text = "The movie was okay."

# Clearly positive reviews (e.g., "This movie was amazing!").

# Clearly negative reviews (e.g., "This movie was terrible!").

# Neutral or ambiguous reviews (e.g., "The movie was okay.").

# Tokenize the input
inputs = tokenizer(input_text, return_tensors="tf", padding=True, truncation=True)

# Load the original (untrained) BERT model
original_model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

# Load the fine-tuned BERT model
fine_tuned_model = TFAutoModelForSequenceClassification.from_pretrained("my_imdb_model")

# Get predictions from the original model
original_outputs = original_model(inputs)
original_probs = tf.nn.softmax(original_outputs.logits, axis=1)

# Get predictions from the fine-tuned model
fine_tuned_outputs = fine_tuned_model(inputs)
fine_tuned_probs = tf.nn.softmax(fine_tuned_outputs.logits, axis=-1)

print(original_outputs)
print(fine_tuned_outputs)

print(original_probs)
print(fine_tuned_probs)

# Print the results
print("Original Model Predictions:")
print(f"Negative: {original_probs[0][0]:.4f}, Positive: {original_probs[0][1]:.4f}")

print("\nFine-tuned Model Predictions:")
print(f"Negative: {fine_tuned_probs[0][0]:.4f}, Positive: {fine_tuned_probs[0][1]:.4f}")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some layers from the model checkpoint at my_imdb_model were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.36300606, -0.30346197]], dtype=float32)>, hidden_states=None, attentions=None)
TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-1.6301904,  1.3103275]], dtype=float32)>, hidden_states=None, attentions=None)
tf.Tensor([[0.48511836 0.5148816 ]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.05018659 0.9498134 ]], shape=(1, 2), dtype=float32)
Original Model Predictions:
Negative: 0.4851, Positive: 0.5149

Fine-tuned Model Predictions:
Negative: 0.0502, Positive: 0.9498


In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score

# Function to evaluate model accuracy
def evaluate_model_accuracy(model, dataset):
    all_preds = []
    all_labels = []

    # Iterate over the dataset
    for batch in dataset:
        # print(batch)
        # Unpack the batch
        inputs = {key: batch[0][key] for key in tokenizer.model_input_names}  # Extract input features
        labels = batch[1]  # Extract labels

        # Get model predictions
        outputs = model(inputs)
        probs = tf.nn.softmax(outputs.logits, axis=-1)  # Apply softmax to get probabilities
        preds = tf.argmax(probs, axis=-1).numpy()  # Convert probabilities to class predictions

        # Store predictions and labels
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

    # Calculate accuracy
    # print(all_preds[:3])
    # print(all_labels[:3])
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# Evaluate original model
original_accuracy = evaluate_model_accuracy(original_model, eval_tf_dataset)
print(f"Original Model Accuracy: {original_accuracy:.4f}")

# Evaluate fine-tuned model
fine_tuned_accuracy = evaluate_model_accuracy(fine_tuned_model, eval_tf_dataset)
print(f"Fine-tuned Model Accuracy: {fine_tuned_accuracy:.4f}")

Original Model Accuracy: 0.4710
Fine-tuned Model Accuracy: 0.8210


Fine Tuning Using Lora Techinque

In [ ]:
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


#Fine-Tuning BERT with LoRA (Using PyTorch)

In [ ]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from peft import LoraConfig, get_peft_model, TaskType
from tqdm import tqdm

#  Load IMDb Dataset
raw_datasets = load_dataset("imdb")

#  Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

#  Tokenization Function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

#  Tokenize Dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Convert tokenized data to PyTorch tensors
def convert_to_tensors(dataset):
    input_ids = torch.tensor(dataset["input_ids"])
    attention_mask = torch.tensor(dataset["attention_mask"])
    labels = torch.tensor(dataset["label"])  # Labels are already integer values (0 or 1)

    return TensorDataset(input_ids, attention_mask, labels)

#  Create Small Training & Evaluation Sets
small_train_dataset = convert_to_tensors(tokenized_datasets["train"].shuffle(seed=42).select(range(1000)))
small_eval_dataset = convert_to_tensors(tokenized_datasets["test"].shuffle(seed=42).select(range(1000)))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# !pip install datasets

In [ ]:
tokenized_datasets["train"]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})

In [ ]:
#  Load Model (BERT with LoRA)
lora_config = LoraConfig(task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
summary(model, dtypes=['torch.IntTensor'])

Layer (type:depth-idx)                                       Param #
BertForSequenceClassification                                --
├─BertModel: 1-1                                             --
│    └─BertEmbeddings: 2-1                                   --
│    │    └─Embedding: 3-1                                   22,268,928
│    │    └─Embedding: 3-2                                   393,216
│    │    └─Embedding: 3-3                                   1,536
│    │    └─LayerNorm: 3-4                                   1,536
│    │    └─Dropout: 3-5                                     --
│    └─BertEncoder: 2-2                                      --
│    │    └─ModuleList: 3-6                                  85,054,464
│    └─BertPooler: 2-3                                       --
│    │    └─Linear: 3-7                                      590,592
│    │    └─Tanh: 3-8                                        --
├─Dropout: 1-2                                               --
├─L

In [ ]:
model = get_peft_model(model, lora_config)  # Apply LoRA
model.to(device)



PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(28996, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default

In [ ]:
summary(model, dtypes=['torch.IntTensor'])

Layer (type:depth-idx)                                                      Param #
PeftModelForSequenceClassification                                          --
├─LoraModel: 1-1                                                            --
│    └─BertForSequenceClassification: 2-1                                   --
│    │    └─BertModel: 3-1                                                  108,347,136
│    │    └─Dropout: 3-2                                                    --
│    │    └─ModulesToSaveWrapper: 3-3                                       3,076
Total params: 108,350,212
Trainable params: 38,402
Non-trainable params: 108,311,810

In [ ]:
#  Dataloaders
batch_size = 8
train_dataloader = DataLoader(small_train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=batch_size)

In [ ]:
#  Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

#  Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}"):
        optimizer.zero_grad()

        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} - Training Loss: {avg_train_loss:.4f}")

# Save the Fine-Tuned Model
model.save_pretrained("bert_lora_finetuned")

#  Evaluation Loop
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Evaluation Accuracy: {accuracy:.4f}")

Training Epoch 1: 100%|██████████| 125/125 [01:07<00:00,  1.85it/s]


Epoch 1 - Training Loss: 0.7024


Training Epoch 2: 100%|██████████| 125/125 [01:08<00:00,  1.82it/s]


Epoch 2 - Training Loss: 0.6949


Training Epoch 3: 100%|██████████| 125/125 [01:10<00:00,  1.77it/s]


...

Epoch 4 - Training Loss: 0.6874


Training Epoch 5: 100%|██████████| 125/125 [01:11<00:00,  1.76it/s]


Epoch 5 - Training Loss: 0.6589


Evaluating: 100%|██████████| 125/125 [00:29<00:00,  4.24it/s]

Evaluation Accuracy: 0.6500


Fine Tuning with Dufferent Varients of LORA technique (Hyperparamters)

#1)LoRA with Higher Ranks and Alpha Tuning

In [ ]:
# LoRA with Higher Rank and Alpha
lora_config_high_rank = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=8, lora_alpha=16, lora_dropout=0.1
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Apply new LoRA config to model
model_high_rank = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
model_high_rank = get_peft_model(model_high_rank, lora_config_high_rank)
model_high_rank.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(28996, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default

In [ ]:
from torchinfo import summary
summary(model_high_rank, dtypes=['torch.IntTensor'])

Layer (type:depth-idx)                                                      Param #
PeftModelForSequenceClassification                                          --
├─LoraModel: 1-1                                                            --
│    └─BertForSequenceClassification: 2-1                                   --
│    │    └─BertModel: 3-1                                                  108,605,184
│    │    └─Dropout: 3-2                                                    --
│    │    └─ModulesToSaveWrapper: 3-3                                       3,076
Total params: 108,608,260
Trainable params: 296,450
Non-trainable params: 108,311,810

In [ ]:
for i in small_eval_dataset:
  print(i)

Streaming output truncated to the last 5000 lines.
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0

In [ ]:
#  Dataloaders
batch_size = 8
train_dataloader = DataLoader(small_train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=batch_size)

In [ ]:
#  Optimizer
optimizer = AdamW(model_high_rank.parameters(), lr=5e-5)

#  Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model_high_rank.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}"):
        optimizer.zero_grad()

        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model_high_rank(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} - Training Loss: {avg_train_loss:.4f}")

# Save the Fine-Tuned Model
model_high_rank.save_pretrained("bert_lora_finetuned")

#  Evaluation Loop
model_high_rank.eval()
correct, total = 0, 0

with torch.no_grad():
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model_high_rank(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Evaluation Accuracy: {accuracy:.4f}")

Training Epoch 1: 100%|██████████| 125/125 [01:13<00:00,  1.69it/s]


Epoch 1 - Training Loss: 0.7001


Training Epoch 2: 100%|██████████| 125/125 [01:12<00:00,  1.71it/s]


Epoch 2 - Training Loss: 0.6897


Training Epoch 3: 100%|██████████| 125/125 [01:13<00:00,  1.71it/s]


...

Epoch 4 - Training Loss: 0.3827


Training Epoch 5: 100%|██████████| 125/125 [01:13<00:00,  1.71it/s]


Epoch 5 - Training Loss: 0.3036


Evaluating: 100%|██████████| 125/125 [00:30<00:00,  4.13it/s]

Evaluation Accuracy: 0.8630


###Pros:

1)Better capacity to adapt to complex tasks.

2)Enhanced ability to capture fine-grained data patterns.

###Cons:

1)Increases training time and model size.

2)Higher memory usage.

#2)LoRA with Different Dropout Rates

In [ ]:
# LoRA with Lower Dropout
lora_config_low_dropout = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=4, lora_alpha=8, lora_dropout=0.05
)

# Apply LoRA with modified dropout
model_low_dropout = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
model_low_dropout = get_peft_model(model_low_dropout, lora_config_low_dropout)
model_low_dropout.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(28996, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (defaul

In [ ]:
from torchinfo import summary
summary(model_low_dropout, dtypes=['torch.IntTensor'])

Layer (type:depth-idx)                                                      Param #
PeftModelForSequenceClassification                                          --
├─LoraModel: 1-1                                                            --
│    └─BertForSequenceClassification: 2-1                                   --
│    │    └─BertModel: 3-1                                                  108,457,728
│    │    └─Dropout: 3-2                                                    --
│    │    └─ModulesToSaveWrapper: 3-3                                       3,076
Total params: 108,460,804
Trainable params: 148,994
Non-trainable params: 108,311,810

In [ ]:
#  Dataloaders
batch_size = 8
train_dataloader = DataLoader(small_train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=batch_size)

In [ ]:
#  Optimizer
optimizer = AdamW(model_low_dropout.parameters(), lr=5e-5)

#  Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model_low_dropout.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}"):
        optimizer.zero_grad()

        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model_low_dropout(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} - Training Loss: {avg_train_loss:.4f}")

# Save the Fine-Tuned Model
model_low_dropout.save_pretrained("bert_lora_finetuned")

#  Evaluation Loop
model_low_dropout.eval()
correct, total = 0, 0

with torch.no_grad():
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model_low_dropout(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Evaluation Accuracy: {accuracy:.4f}")

Training Epoch 1: 100%|██████████| 125/125 [01:14<00:00,  1.68it/s]


Epoch 1 - Training Loss: 0.6948


Training Epoch 2: 100%|██████████| 125/125 [01:12<00:00,  1.71it/s]


Epoch 2 - Training Loss: 0.6707


Training Epoch 3: 100%|██████████| 125/125 [01:12<00:00,  1.72it/s]


...

Epoch 4 - Training Loss: 0.4226


Training Epoch 5: 100%|██████████| 125/125 [01:13<00:00,  1.71it/s]


Epoch 5 - Training Loss: 0.3444


Evaluating: 100%|██████████| 125/125 [00:30<00:00,  4.12it/s]

Evaluation Accuracy: 0.8580


###Pros:

1)Lower dropout may retain more information, increasing accuracy.

2)Higher dropout can prevent overfitting with small datasets.

###Cons:

1)Very low dropout may lead to overfitting.

2)High dropout may harm performance if too aggressive.

#3)LoRA with Bias-only Tuning

In [ ]:
from peft import LoraConfig, TaskType

# Define LoRA configuration with bias-only tuning
lora_config = LoraConfig(
    r=1,  # kept very low
    lora_alpha=1,
    target_modules=["query", "key", "value", "dense"],  # Specify target modules
    bias="all",  # Tune all bias parameters
    task_type=TaskType.SEQ_CLS,  # Task type (e.g., sequence classification)
)


base_model4 = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
# Apply the configuration
model4 = get_peft_model(base_model4, lora_config)
model4.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(28996, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_featur

text
label
input_ids
token_type_ids
attention_mask

In [ ]:
from torchinfo import summary
summary(model4, dtypes=['torch.IntTensor'])

Layer (type:depth-idx)                                                      Param #
PeftModelForSequenceClassification                                          --
├─LoraModel: 1-1                                                            --
│    └─BertForSequenceClassification: 2-1                                   --
│    │    └─BertModel: 3-1                                                  108,477,696
│    │    └─Dropout: 3-2                                                    --
│    │    └─ModulesToSaveWrapper: 3-3                                       3,076
Total params: 108,480,772
Trainable params: 271,874
Non-trainable params: 108,208,898

In [ ]:
#  Dataloaders
batch_size = 8
train_dataloader = DataLoader(small_train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=batch_size)

In [ ]:
#  Optimizer
optimizer = AdamW(model4.parameters(), lr=5e-5)

#  Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model4.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}"):
        optimizer.zero_grad()

        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model4(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} - Training Loss: {avg_train_loss:.4f}")

# Save the Fine-Tuned Model
model4.save_pretrained("bert_lora_finetuned")

#  Evaluation Loop
model4.eval()
correct, total = 0, 0

with torch.no_grad():
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model4(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Evaluation Accuracy: {accuracy:.4f}")

Training Epoch 1: 100%|██████████| 125/125 [01:22<00:00,  1.52it/s]


Epoch 1 - Training Loss: 0.7038


Training Epoch 2: 100%|██████████| 125/125 [01:21<00:00,  1.53it/s]


Epoch 2 - Training Loss: 0.6929


Training Epoch 3: 100%|██████████| 125/125 [01:21<00:00,  1.53it/s]


...

Epoch 4 - Training Loss: 0.5631


Training Epoch 5: 100%|██████████| 125/125 [01:21<00:00,  1.53it/s]


Epoch 5 - Training Loss: 0.3997


Evaluating: 100%|██████████| 125/125 [00:33<00:00,  3.76it/s]

Evaluation Accuracy: 0.8420


###Pros:

1)Very lightweight, minimal storage and computational cost.

2)Effective for small datasets or limited hardware.

###Cons:

3)Limited expressivity for complex tasks